In [16]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [17]:
filename = "../../data/03/ABox.ttl"

In [18]:
text1 = '''CQ_3.1
Return all the risks with type "Cumulative process", the asset they are assiged to, the layers they exist within, and the documents that document them.
'''

query1 = '''
PREFIX hero: <http://purl.org/sirius/ontology/hero/>

SELECT ?risk ?asset ?layer ?document
WHERE {
  ?risk_assessment a hero:RiskAssessment ;
                  hero:assignsRisk ?risk ;
                  hero:assignsRiskTo ?asset .
  ?risk hero:withType hero:cumulative ;
        hero:withinLayer ?layer ;
        hero:isDocumentedBy ?document .
}
'''

In [19]:
text2 = '''CQ_3.2
Return all the risks existing with the layers "Region" or "Site", the asset they are assiged to, their type, and the start and end dates of the time intervals they exist in.
'''

query2 = '''
PREFIX hero: <http://purl.org/sirius/ontology/hero/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX tvc: <http://www.essepuntato.it/2012/04/tvc/>

SELECT ?risk ?asset ?type ?time_interval_start ?time_interval_end
WHERE {
    ?risk_assessment a hero:RiskAssessment ;
                    hero:assignsRisk ?risk ;
                    hero:assignsRiskTo ?asset .
    ?risk hero:withType ?type ;
          hero:withinLayer ?layer ;
          tvc:atTime ?time_interval .
    ?time_interval ti:hasIntervalStartDate ?time_interval_start ;
                  ti:hasIntervalEndDate ?time_interval_end . 
    FILTER ( ?layer = hero:region || ?layer = hero:site )
}
'''

In [20]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_3.1
Return all the risks with type "Cumulative process", the asset they are assiged to, the layers they exist within, and the documents that document them.

+-------------------------------------------------+----------------------------------------------------+-----------------------------------------------+----------------------------------------------------+
| risk                                            | asset                                              | layer                                         | document                                           |
|-------------------------------------------------+----------------------------------------------------+-----------------------------------------------+----------------------------------------------------|
| http://purl.org/sirius/ontology/data/03/risk-01 | http://purl.org/sirius/ontology/data/03/baptistery | http://purl.org/sirius/ontology/hero/building | http://purl.org/sirius/ontology/data/03/document-a |
| http://purl.or